[Oregon Curriculum Network](http://www.4dsolutions.net/ocn) <br />
[Discovering Math with Python](Introduction.ipynb)

# Generating the FCC

The Face Centered Cubic lattice is equivalently the CCP and the IVM the way we're thinking of it here.

IVM = Isotropic Vector Matrix which is what Buckminster Fuller dubbed it, drawing attention to the same-length segments between any sphere center and it's twelve neighbors.

Shown below are the twelve neighbors of a nuclear sphere at the center.

!["freq1"](freq1.png)

My technique for generating the above is to add all permutations of {2, 1, 1, 0} to the nuclear (0, 0, 0, 0).

The itertools.permutations algorithm treats the two 1s as distinct and so creates more permutations than we need.

Those permutations are:

In [1]:
from itertools import permutations
g = permutations((2,1,1,0))
unique = {p for p in g}  # set comprehension
print(unique)

{(0, 1, 1, 2), (1, 0, 1, 2), (2, 0, 1, 1), (0, 2, 1, 1), (0, 1, 2, 1), (1, 2, 1, 0), (1, 1, 2, 0), (2, 1, 1, 0), (1, 0, 2, 1), (1, 2, 0, 1), (2, 1, 0, 1), (1, 1, 0, 2)}


What we're talking about here are ["Quadrays"](https://en.wikipedia.org/wiki/Quadray_coordinates) i.e. vectors built from linear combinations of four basis rays from the center of a regular tetrahedron to its four corners.  These have the advantage of assigning integer coordinates to all FCC lattice points.

The system for getting successive layers is to add the twelve unique vectors above, but only keep those not already in the current layer, or in the previous layer.  In other words, we force the database to enlarge only in the direction of new unique spheres.  Every sphere in the current layer is the source of twelve new ones, but only a few of those will be kept, and added to the next layer.

I have the ability to convert Quadrays to XYZ vectors.  Once the algorithm has located all the balls in the next layer, I get the XYZ equivalents as no rendering software has any native understanding of Quadrays.

In [2]:
import qrays

nucleus = {qrays.Qvector((0,0,0,0))}

layer1 = tuple(map(qrays.Qvector, 
         ((0, 1, 1, 2), (1, 0, 1, 2), 
          (2, 0, 1, 1), (0, 2, 1, 1), 
          (0, 1, 2, 1), (1, 2, 1, 0), 
          (1, 1, 2, 0), (2, 1, 1, 0), 
          (1, 0, 2, 1), (1, 2, 0, 1), 
          (2, 1, 0, 1), (1, 1, 0, 2))))

def next_layer(curr_layer, prev_layer):
    next_layer = set()
    for qv in curr_layer:
        for bv in layer1:
            v_sum = qv + bv
            if (not v_sum in curr_layer 
                and not v_sum in prev_layer):
                next_layer.add(v_sum)
    return next_layer

nl   = next_layer(nucleus, nucleus) # 1-freq
nnl  = next_layer(nl, nucleus)      # 2-freq
nnnl = next_layer(nnl, nl)          # 3-freq
print("Number of balls in successive layers:")
print(len(nl))
print(len(nnl))
print(len(nnnl))

Number of balls in successive layers:
12
42
92


The final step involves generating scene description language and interpolating the resulting instructions into a POV-Ray script.

In [3]:
sph = """sphere { %s 0.5 texture { pigment { color rgb <1,0,0> } } }"""
    
def get_xyz(qvectors):
    xyz_vectors = []
    for qv in qvectors:
        xyz_vectors.append(qv.xyz())
    return xyz_vectors

def make_spheres(xyz_vectors):
    pov = open("fragment.pov", "w")
    for xyzv in xyz_vectors:
        s = sph % "<{0.x}, {0.y}, {0.z}>".format(xyzv)
        print(s, file=pov)
        
vs = get_xyz(nl)
make_spheres(vs)

Here's what scene description language looks like:

<pre>
sphere { &lt;0.0, -0.7071067811865475, -0.7071067811865475&gt; 
         0.5 texture { pigment { color rgb &lt;1,0,0&gt; } } }
sphere { &lt;0.0, -0.7071067811865475, 0.7071067811865475&gt; 
         0.5 texture { pigment { color rgb &lt;1,0,0&gt; } } }
</pre>

...and so on.

Two frequency:

[ ](test_layer2.png)

!["freq2"](freq2.png)

Three frequency.

!["freq3"](freq3.png) 